In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from scipy import stats
from itertools import product

## part a)

In [3]:
index_list1 = ['SEQN','RIDAGEYR', 'RIDRETH3', 'DMDEDUC2', 'DMDMARTL',
              'RIDSTATR', 'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR']

# read files and and a column
path1 = r"./demographic"
files1 = os.listdir(path1)
dat1 = []
for file in files1:
    if file.endswith('.XPT'):
        a = pd.read_sas(path1+'/'+file)
        # select specified columns
        a = a[index_list1]
        # add the column identifying to which cohort each case belongs
        a['YEAR'] = [file[6:10] for i in range(len(a))]
        dat1.append(a)
        
df1 = pd.concat(dat1, ignore_index=True)

# rename columns
df1 = df1.rename(columns = {'SEQN': 'id','RIDAGEYR':'age', 
                            'RIDRETH3':'race', 'DMDEDUC2':'education', 
                            'DMDMARTL':'marital_status', 'SDMVPSU':'psu', 
                            'SDMVSTRA':'stratum'})
df1.columns = df1.columns.str.lower()

# object: id
# int: age, psu, stratum
# category: gender, race, education, marital_status, ridstatr
df1['id'] = df1['id'].astype(int).astype(object)

l1_1 = ['age', 'psu', 'stratum']
df1[l1_1] = df1[l1_1].astype(int)

l1_2 = ['race', 'education', 'marital_status', 'ridstatr']
df1[l1_2] = df1[l1_2].fillna(-1).astype('Int8')
for k in l1_2:
    df1[k] = pd.Categorical(df1[k])

In [4]:
# columns
dat = pd.read_sas('./ohxden/OHXDEN_G2011.XPT')
index_list2 = ['SEQN','OHDDESTS']
all_index = dat.columns
for a in all_index:
    if a[:3] == 'OHX' and a[-2:] == 'TC':
        index_list2.append(a)

path2 = r"./ohxden"
files2 = os.listdir(path2)
dat2 = []
for file in files2:
    if file.endswith('.XPT'):
        a = pd.read_sas(path2+'/'+file)
        # select specified columns
        index_list2 = ['SEQN','OHDDESTS']
        all_index = dat.columns
        for idx in all_index:
            if idx[:3] == 'OHX' and idx[-2:] == 'TC':
                index_list2.append(idx)
        a = a[index_list2]
        # add the column identifying to which cohort each case belongs
        a['YEAR'] = [file[-8:-4] for i in range(len(a))]
        dat2.append(a)
        
df2 = pd.concat(dat2, ignore_index=True)

# rename columns
df2 = df2.rename(columns = {'SEQN': 'id'})
df2.columns = df2.columns.str.lower()

# object: id
# category: ohddests, ohxXXtc, ohxXXctc
columnNames2 = df2.columns

## seqn
df2['id'] = df2['id'].astype(int).astype(object)

## ohxXXctc
l2_1 = [x for x in columnNames2 if x[-3:] == 'ctc']
for idx in l2_1:
    df2[idx] = df2[idx].str.decode('utf8', errors='strict').astype('category')

## ohxXXtc & ohddests
l2_2 = [x for x in columnNames2 if x[-2:]=='tc' and x[-3]!='c']
l2_2.append('ohddests')
df2[l2_2] = df2[l2_2].fillna(-1).astype(int)
for x in l2_2:
    df2[x] = pd.Categorical(df2[x])

In [ ]:
# save to pickle
df1.to_pickle('./demographic.pkl')
df2.to_pickle('./ohxden.pkl')

## part b)

In [ ]:
# merge two tables
demo = df1.merge(df2[['id', 'year', 'ohddests']], on=['year', 'id'], how='left')
demo.rename(columns={'ridstatr': 'exam_status', 'ohddests': 'ohx_status'}, inplace=True)

# add a new column 'under_20'
demo['under_20'] = 'no'
demo.loc[demo['age'] < 20, 'under_20'] = 'yes'

# add a new column 'college'
conditions = [
    (demo['education'] == 4) | (demo['education'] == 5),
    (demo['education'] != 4) & (demo['education'] != 5),
    (demo['age'] < 20)
]
values = ['some college/college graduate', 'No college/<20', 'No college/<20']
demo['college'] = np.select(conditions, values)

# create a new dataframe
demo_new = demo[['id', 'gender', 'age', 'under_20', 'college', 'exam_status', 
                 'ohx_status']]

# create a new variable called 'ohx'
demo_new['ohx'] = 'missing'
demo_new.loc[(demo_new['exam_status'] == 2) & (demo_new['ohx_status'] == 1), 'ohx'] = 'complete'
demo_new[['under_20', 'college', 'ohx']] = demo_new[['under_20', 'college', 'ohx']].astype('category')

demo_new

## part c)

In [ ]:
demo_new.drop(demo_new[demo_new['exam_status'] != 2].index, inplace=True)
demo_new

There are 1757 rows removed from the table, and 37399 subjects remaining.

## part d)

In [ ]:
def count_summ(varname):
    """
    Count the number of each variable

    Parameters
    ----------
    varname : string
        The variable name.
        
    Returns
    -------
    The table of the results.

    """
    summ_tbl = (demo_new
        .groupby([varname, 'ohx'])
        .apply(lambda x: len(x))
        )
    summ_tbl.index.names = ['category', 'ohx']
    summ_tbl = summ_tbl.reset_index().pivot(index='category', columns='ohx')
    summ_tbl.columns = ['complete', 'missing']
    p = stats.chi2_contingency(np.array(summ_tbl))[1]
    summ_tbl['p-value'] = ""
    summ_tbl.iloc[0, 2] = ("%.3g" % p)
    string = "n = {0:d} ({1:4.2f}%)"
    for i in range(2):
        t = sum(summ_tbl.iloc[i,0:2])
        for j in range(2):
            summ_tbl.iloc[i,j] = string.format(int(summ_tbl.iloc[i,j]), 
                                               100*summ_tbl.iloc[i,j]/t)
    return summ_tbl

In [ ]:
# count the number of `under_20`, `gender`, and `college`
under20_tbl = count_summ('under_20')
gender_tbl = count_summ('gender')
college_tbl = count_summ('college')
tbl_all = pd.concat([under20_tbl, gender_tbl, college_tbl])

# count the number of `age`
age_tbl = (demo_new
           .groupby('ohx')
           .apply(lambda x: pd.Series({'mean': np.mean(x.age),
                                       'sd': np.std(x.age)})))
tbl_all.loc['age_mean',:] = ''
tbl_all.iloc[6,0] = 'mean = {0:4.2f} (sd = {1:4.2f})'.format(age_tbl.iloc[0,0], 
                                                             age_tbl.iloc[0,1])
tbl_all.iloc[6,1] = 'mean = {0:4.2f} (sd = {1:4.2f})'.format(age_tbl.iloc[1,0], 
                                                             age_tbl.iloc[1,1])
p =  stats.ttest_ind(demo_new.loc[demo_new['ohx']=='complete', 'age'],
                      demo_new.loc[demo_new['ohx']!='complete', 'age']).pvalue
tbl_all.iloc[6,2] = ("%.3g" % p)

# polish the table
a1 = np.append(np.repeat(['under_20', 'gender', 'college'],2), ['age'])
a2 = ['no', 'yes', 'male', 'female', 'no college/<20', 'some college/college graduate',
      'age_mean']
tbl_all.index = pd.MultiIndex.from_arrays([a1, a2], names=['index', 'category'])
tbl_all